In [1]:
include("refactored_bo_3d.jl")

(-2.0556423689449357, 109)


In [2]:
using BenchmarkTools

In [3]:
x = range(-10, 10, length = 100)
y = range(-10, 10, length = 100)
X = repeat(x, inner = length(y))
Y = repeat(y, outer = length(x))
XY = hcat(X,Y)
θ = [10.0, 1.0] # Hyperparameters in the form of (σ, ℓ) or (σ, ℓ, p)
rbf = squared_exponential(θ)
num_init_samples = 10
BUDGET = 200

200

In [4]:
@benchmark eval_KXX(rbf, XY, 1e-6)

LoadError: UndefVarError: `eval_KXX` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [4]:
opt, f = cross_in_tray(X, Y)
num_init_samples = 10 # Initial samples
σ = 1e-6 # Noise variable
𝒟 = rand_sample(XY, num_init_samples, f, σ)

10×3 Matrix{Float64}:
 -6.36364    9.19192  -1.03566
 -6.76768    4.34343  -1.5688
 -0.505051   2.92929  -1.59475
  6.9697     5.9596   -1.40156
  8.38384    7.57576  -1.5088
 -8.9899    -6.76768  -1.30817
  1.51515   10.0      -1.50183
  6.76768    0.10101  -1.30784
 -2.92929    5.15152  -1.54547
 -6.36364   -1.11111  -1.37844

In [26]:
@benchmark eval_kxx(rbf, 𝒟[:, 1:2], 200, 1e-6)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  11.700 μs …  21.718 ms  ┊ GC (min … max):  0.00% … 99.53%
 Time  (median):     15.300 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   52.506 μs ± 441.786 μs  ┊ GC (mean ± σ):  34.48% ±  7.15%

  █▇▅▃▁▁▁▁▁        ▂▄▂▁▁▂▃▃▂▁▁▁▁                               ▁
  █████████▇▆▄▅▅▄▄▅█████████████████▇▇▇▆▆▆▆▅▆▅▆▅▅▆▄▄▅▅▅▄▄▄▄▅▄▄ █
  11.7 μs       Histogram: log(frequency) by time       186 μs <

 Memory estimate: 312.88 KiB, allocs estimate: 7.

In [27]:
@benchmark eval_KxX(rbf, 𝒟[:, 1:2], XY, 200)

BenchmarkTools.Trial: 724 samples with 1 evaluation per sample.
 Range (min … max):  4.400 ms … 30.118 ms  ┊ GC (min … max):  0.00% … 75.52%
 Time  (median):     7.023 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.892 ms ±  2.471 ms  ┊ GC (mean ± σ):  20.92% ± 18.60%

  █▅▂            ▃                                            
  ███▇▄▄▅▄▄▄▃▂▁▃▆█▆▇▅▆▅▅▅▃▄▄▃▃▃▃▃▂▂▂▂▂▁▃▂▃▂▂▁▂▂▂▂▁▂▁▂▂▂▁▂▂▁▂ ▃
  4.4 ms         Histogram: frequency by time          15 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 7.

In [69]:
@benchmark main()

(-2.032559795799709, 132)
(-2.0592181446514584, 29)
(-2.0592181446514584, 129)


BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 5.759 s (9.57% GC) to evaluate,
 with a memory estimate of 2.44 GiB, over 11106 allocations.

In [5]:
begin
    Κ_ss = eval_KXX(rbf, XY, 1e-6)
    Κ_xx = eval_kxx(rbf, 𝒟[:, 1:2], 200, 1e-6)
    Κ_xs = eval_KxX(rbf, 𝒟[:, 1:2], XY, 200)
    μ_pri = mean_zero()
    GP = GaussianProcess(μ_pri, rbf.kernel, Κ_ss, Κ_xx, Κ_xs)
end

GaussianProcess{var"#m#9", typeof(squared_exponential_κ), Matrix{Float64}}(var"#m#9"(), Main.squared_exponential_κ, [100.000001 97.98007140447176 … 1.066768466969628e-170 1.9151695967140056e-172; 97.98007140447176 100.000001 … 5.704382041783347e-169 1.066768466969628e-170; … ; 1.066768466969628e-170 5.704382041783347e-169 … 100.000001 97.98007140447176; 1.9151695967140056e-172 1.066768466969628e-170 … 97.98007140447176 100.000001], [100.000001 0.000724262399455 … 0.0 0.0; 0.000724262399455 100.000001 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.4022966208023336e-81 6.634448564912398e-80 … 5.957007376150902e-57 5.164073312841352e-57; 1.1393845513696354e-45 2.0241716985444464e-44 … 3.071792210234945e-66 9.999267201175847e-67; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [8]:
function main_loop(GP, 𝒟, XY, f, σ)
    for i in num_init_samples : BUDGET
        exp_imp, μ_post = expected_improvement(GP, 𝒟, XY)
        𝒟 = best_sampling_point(exp_imp, XY, 𝒟, f, σ)
        if i != BUDGET
            GP.Κ_xx = update_KXX!(rbf, GP.Κ_xx, i, 𝒟[:, 1:2], 1e-6)
            GP.Κ_xs = update_kxX!(rbf, GP.Κ_xs, i, 𝒟[:, 1:2], XY)
        end
    end
end

main_loop (generic function with 1 method)

In [9]:
@benchmark main_loop(GP, 𝒟, XY, f, σ)

BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  2.831 s …    3.272 s  ┊ GC (min … max): 4.38% … 5.10%
 Time  (median):     3.052 s               ┊ GC (median):    4.76%
 Time  (mean ± σ):   3.052 s ± 311.647 ms  ┊ GC (mean ± σ):  4.76% ± 0.51%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.83 s         Histogram: frequency by time         3.27 s <

 Memory estimate: 1.68 GiB, allocs estimate: 12091.